# CS 470 - Semester Project

## LyricaLlama: A fintuned LLM for generating lyrics


### Fall 2023

### Grant Lewis

-----

### Hours:

| Date      | Duration    | Type            | Description |
|-----------|-------------|-----------------|-------------|
| 11/3/2023 | 1.5 hours   | Research        | Found song data to use in fine-tuning the LLM. |
| 11/21/2023| 1 hour      | Research        | Researched current best approaches to fine-tune an LLM with limited resources. Decided to use Q-LoRA. |
| 12/5/2023 | 1 hour      | EDA             | Conducted experiments on the data to better understand what I was working with. |
| 12/6/2023 | 2.5 hours   | Data Preparation| Prepared the data for use in finetuning. |
| 12/6/2023 | 3.5 hours   | MLOPs           | Prepared and setup a new environment for the finetuning the LLM. Ran into several issues with versioning that had to be resolved. |
| 12/7/2023 | 4.5 hours   | Development     | Built out finetuning scripts and tested it on a small sample of data. |
| 12/7/2023 | 3 hours     | Debugging       | Attempted to start training at scale. Had issues that needed to be debugged. Also added more checkpoints to allow for recovery from crashing |
| 12/7/2023 | 1 hour      | Training        | Monitored the first iteration of training at scale. Got to a checkpoint at 5% before crashing. |
| 12/8/2023 | 1 hour      | Training        | Process failed overnight. Restarted training and monitored for a while. |
| 12/9/2023 | 0.5 hours   | Training        | Process failed overnight. Restarted training and monitored for a while. |
| 12/9/2023 | 1 hour      | Testing         | Set up and conducted some tests to see if there were improvements. |
| 12/11/2023| 2 hours     | Testing         | Finished Conducting Tests. |
| **Total Time**| **21.5 hours**  |                 |             |


### Purpose
My goal for this project was to fine-tune a large language model (LLM) on lyrics from songs to enhance its creative output in generating song lyrics. Recognizing that LLMs often replicate what they're trained on without genuine creativity, I aimed to investigate whether fine-tuning could yield more original and contextually appropriate lyrics.

### Preparation
For this task, I chose the dataset from [kaggle](https://www.kaggle.com/datasets/neisse/scrapped-lyrics-from-6-genres/) and the [meta-llama/Llama-2-7b-hf](https://huggingface.co/meta-llama/Llama-2-7b-hf) model from Hugging Face. My selection was driven by the dataset's large, diverse collection of songs and the model's balance between size and capability.

### Data Preparation and Development
The dataset contained a strong skew towards Portuguese music, due to the data being collected from a Brazilian website. I opted to focus exclusively on the English subset, which still accounted for more songs than any other language, totaling 191,814 songs.  I was hoping to use the full dataset in training and testing, however, due to hardware constraints, I ended up using only 25% of English songs, resulting in a training set of 34,667 songs.  I felt that this size provided sufficient information for the model to learn from while still being manageable for how long it would take to train.

Technical difficulties emerged during the setup phase, particularly regarding dependency versioning conflicts involving bits-and-bytes. I am by no means an expert at MLOps and versioning, so resolving these issues required many iterations and a significant amount of troubleshooting and experimentation for me.  This resulted in me taking a deep dive into compatibility specifics.  However, I was able to learn a lot along the way, including how to better troubleshoot issues arrising from managing coplicated python environments that will be useful for me in future projects.

Part of the preparation required me to develop a solid prompt format for training the model.  I tested several different prompts and instructions, but ended up using the provided instructions and formatting since they seemed to work well with the base Llama model.  The prompt format is as follows:
```
### Instruction: You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided.

### Input:
Write lyrics for a song titled "{song_name}" to be performed by {artist}[ using the following geners: {comma delimited list of genres}].

### Response:
{lyrics}
```
Note that the portion of text in square brackets was not included when no genres were provided.

### Training
I ran an initial small-scale training run to validate the training pipeline worked properly and that the model was being saved properly. Once I was happy with the results, I moved to full-scale training on the full trainingset.  However, I once again encountered unforeseen issues when scaling up for training. The process ended up hanging when loading data onto the GPU due to an increase in batch size. To mitigate the problem, I had to adjust the batch size slightly. I finetuned the model on the training set for 2 epochs over the course of three days.  Unfortunately there were several intermitent interruptions due to the server ending the process (not from run-time errors).  Fortunately, I had increased checkpointing during training so I was able to pick up from a recent checkpoint despite the crashes.  After training was completed, I uploaded the model to Hugging Face as [grantsl/LyricaLlama](https://huggingface.co/grantsl/LyricaLlama).

### Results
The testing phase, though short, was illuminating. I compared the outputs of the original Llama-2-7b and the finetuned LyricaLlama models for 6 different songs. 3 of the songs existed before the training data was collected and 3 new songs released this year. The assessment was based on subjective rankings by various individuals. The results leaned towards the finetuned model, indicating an improvement in lyric generation quality.  In all but one case, the finetuned model produced more coherent lyrics which the majority of people favored.  In the one instance where this wasn't the case, there was actually a tie between the two models.

In looking over the results of LyricaLlama, it can also be seen how it learned to keep lines shorter, and often make solid attempts at rhyming.  This is impressive to see, especially given the limited amount of time it was trained for.

It is also important to note that both models occasionally produced explicit language, reflecting the unfiltered nature of the training dataset. This highlights the impact of dataset composition on model output, an important consideration for future projects.

| Song_title               |   Favored_Base (Summed) |   Favored_Lyrical (Summed) | Index with Most Votes |  Index with Least Votes |
|:-------------------------|---------------:|------------------:|-------:|--------:|
| All I Want For Chirstmas |              1 |                 9 |      3 (Lyrical) |       0 (Base) |
| Blinding Lights          |              4 |                 6 |      5 (Lyrical) |       0 (Base) |
| Lover                    |              3 |                 7 |      4 (Lyrical) and 5 (Lyrical) |       0 (Base) |
| Electric Touch           |              2 |                 8 |      5 (Lyrical) |       0 (Base) |
| Next Thing You Know      |              4 |                 6 |      5 (Lyrical) |       1 (Base) |
| Penthouse                |              5 |                 5 |      1 (Base) and 5 (Lyrical) |       3 (Lyrical) |

![results](./results_plot.png)

### Lessons Learned and Future Work
This project was a great learning experience for me to better understand how to manage and correct issues in python environments as well as better understanding how LLM's work and how to fine-tune them.

In future iterations, I would explore varying the dataset composition further, perhaps by incorporating more variety in the instructions and format of the prompts.  I would also like to compare how different artists and genres impact the lyrics which the model produces. This could reveal more about the model's adaptability and creative range. Additionally, I would invest more time in the testing phase to gain deeper insights into the model's performance improvements.

### Conclusion
This project successfully demonstrated that fine-tuning an LLM on a specific dataset can enhance its performance in the creative task of generating lyrics, even if some of the lines were taken from other songs, and the model occassionally rambled.  However, I could see how this model could be used to spitball different ideas and come up with new ideas for additions to lyrics, even if the produced lyrics don't always meld well together.  I am extremely satisfyied with the results and that I was able to successfully finetune an LLM despite technical challenges.  It is also very satisfying to know that the model is on Hugging Face and available for others to be able to download and be able to play with. This project has helped solidify my experience with LLM's to improve my understanding and competency in the field of creative AI.

#### Test Results in Json Format:


In [ ]:


{
    "All I Want For Christmas": {
        "is_new": false,
        "song": "All I Want For Christmas",
        "artist": "Mariah Carey",
        "genres": [
            "Christmas"
        ],
        "results": [
            {
                "model_name": "meta-llama/Llama-2-7b-hf",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"All I Want For Christmas\" to be performed by Mariah Carey using the following genres: Christmas.\n\n### Lyrics:\n",
                "responses": "<p>All that I want for Christmas is you</p><p></p>\r\n\r\nAll you want is to see me happy\r\rAnd all that you got is a wish for me\rAll the gifts I could ever want are right under the tree\rBut the only thing I really want, I'll never see\rThat's all I wanna do, all the time\rYou're all alone, baby, no one'd care if you cried\rSo all you need is me, here with you every night\rI'm all about you, you've been all around the world and back\rOh, oh, the best thing about Christmas, is all of the lights and the snow\rIt'a time for sharing, a time of giving, it' a holiday so special to me and to you\rA time to hold each other, to love eachother, for all we can be\rWhen the magic of Christmas comes, we'l be here, just you and me",
                "is_finetuned": false
            },
            {
                "model_name": "meta-llama/Llama-2-7b-hf",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"All I Want For Christmas\" to be performed by Mariah Carey using the following genres: Christmas.\n\n### Lyrics:\n",
                "responses": "- I don't want a lot for Christmas\nThere's just one thing I need\nI don\u2019t care about the presents\nUnderneath the tree\nJust as long as you are there\nWith me.",
                "is_finetuned": false
            },
            {
                "model_name": "meta-llama/Llama-2-7b-hf",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"All I Want For Christmas\" to be performed by Mariah Carey using the following genres: Christmas.\n\n### Lyrics:\n",
                "responses": "*All i want for christmas is you*\nAll you want is me?\nI don't want you, I don\u2019t need you\nYou're not my friend, you've been so unkind\nIf you wanna be my lover you gotta get with my friends\nMake it stop, make it go away\nThis can' all be undone\nAnd I'll be happy if you just go\n'Cause all I want, all i need\nIs you out of my life\nThere's a war going on inside my head\nTrying to figure out what I should do\nWith you in my heart and you on my mind\nIt'd be so much easier if I didn'\nTell me what you really want from me\nBut if all you can offer me is pain\nThen I can live without it\nSo go on, take your love and leave me alone\nCuz all that I need is to know\nThat you don\u00b4t love me, anymore\n",
                "is_finetuned": false
            },
            {
                "model_name": "grantsl/LyricaLlama",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"All I Want For Christmas\" to be performed by Mariah Carey using the following genres: Christmas.\n\n### Lyrics:\n",
                "responses": "All i want for christmas is you\nIs you, baby\nI don't want a lot for chritmas\nThere's just one thing I need\nOh, i don;t care about the presents\nUnderneath the christman tree\nFor christmastime, I just want to hold\nYou, babe\nAnd watch you trimming the tree. (tree, tree,tree)\nYeah, yeah, oh, no, uh\nWhat more can I say?\nAfter all the vows that have been spoken\nMaybe I really do need you here with me\nBaby, all I want, for Christmas is\n(All,all,yeah) you. Yeah, you baby,\nJust for tonight, we'll make it right\nWe're gonna celebrate the holiday\nWith carols and christian love\nTonight\nWell, it'd be a perfect Christmas\nIf you were here to celebration\nSo darling, if you really love me, (love me) oh\nCome home for the Christmas, Christmas baby. Oh\nPlease come home\n'Cause I, 'cause i\nWant you home, home toniiight. Toniiiiiiiight!\nOoh, ooh. Ooh-ooh\nChristmas time, time is here\nHappiness and cheer\nFun for all that children call\nTheir favorite time of year\nSnowflakes in your hair\nCarols everywhere\nReindeers and sleighs\nSing Christmas carol\nSilver bells, ringing\nStrings of lights, on the trees\nThat' s what Christmas means to me. To meeeee\nTo me! Oh,oh! Yeah\nA Christmas to remember\nDarling you and me together\nIt' ll be the perfect memory\nOf all those Christmas' long ago\nBut this Christmas now will be\nthe best one ever\nAs long as you' re here, here by my side\nNow, darlin'\nDo you hear what I hear? (hear, hear) Oh-ohh-hooo\nNoel, Noel\nKing Jesus is born\nMerry Christmas everyone\nJoy to the world\nLet'em know that you are here toniiiiggghhht! (toniiigghht) Yeah! Oooohhh\nYes, yes, yeaahh! Yea, whooaaahhh! Yes! Uh, huh\nWhooah! Whooaaaaaaa! Wooooahaa! Whooo! Ho! Hoo! Yeeeaaaeeeah\nAh, ah, ho, hey, yeeaaaa, haaay\nUh uhh, aah uuh, aaah aaheeeey\nMmmmmm, mmm, mm, wooaohhoo\nHeeey, ow! Hey,ho! Hee-hee! Mmm! Ahhhh..\nEeehheey! Eeehey! Aah-ahahhaaaheyyy\nFuhhh-haa-hahheyeaaayyy\nThis Christmas I've got you with\nMeeeehhehhaeeeaahyyyeeyyeehhhhheheehhyeeeayyyeehhhhaaaaaayyeeha\nGeez, geez,",
                "is_finetuned": true
            },
            {
                "model_name": "grantsl/LyricaLlama",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"All I Want For Christmas\" to be performed by Mariah Carey using the following genres: Christmas.\n\n### Lyrics:\n",
                "responses": "I don't want a lot for Christmas\nThere is just one thing I need\nChristmas carols and church bells\nMistletoe hung where you walk\nUnderneath the mistle toe\nHappy holidays the family's there\nThat'll be the time you're here\nWith every Christmas card I write\nMay your days be merry and bright\nAnd may all your Christmases be white\nAll the lights are shining so brightly\nWonderful Christams\nSanta Claus is coming to town\nHe' s making a list\nChecking it twice\nGonna find out who'se naughty or nice\nWell I know one lucky person\nWho' se gonna get a gift on Christmas day\nIt'S me\nYeah, me, yeah\nMe, myself and I\nOh, I don 't need to hang mistlettoh\nUp on the ceiling\nJust as long as I got my baby\nStanding right beside me. Oh\nLittle baby Jesus, born on christmas day,\nA gift for all the world to celebrate\nSo I'm wishing you a merrry christams, baby. Yeah\nEveryone' 's waiting for santa claus\nTo make their christas bright, bright. Bright\nWaiting for christasm morning\nFirst thing on your list, you know you gotta have\nSomeone to hold. Hold you tight\nYou've got to have some one to kiss\nOn christam morning. Whoo\nWhen you wish upon a star\nYour christsm dreams come true\nIf you really believe\nThen you can do anything\nYes, anything you want\nAnything you need, need this christmass\nNow I Don' t Need To Hang Mistletow\nUpon The Ceiling, Just As Long As I Got My Baby\nBy My Side\n'Cause I Just Wanna Spend Christmas With You\nBaby, Baby, Oh Baby. Baby!\nMerry Christmas, everybody! Oh yeah! Merry\nChrismas! Yeah! Happy holiday! Christmas! Whoo! Ho Ho! Hoo\nOoh, ooh! Ooh Oooooh\n(Oh Baby)\n[Guitar Solo]\nThe christmastree' ll be\nRight there in the square\nSing your christma songs\nFill the air\nPut christman in your heart\nEv'rywhere\nTake some christmans cheer\nFor your friends and family\nLet them know that you care\nThats the christms spirit\nWhich will never die\nCelebrate the birth of christ\nShare the spirit of the season\nMake christmus bright for everyone\nSpread the word, let it be heard\nWe' re gon' celebration the joy\nOf christus birth\nHave a very, very merriest christmaz\nThis christmay\nSee you next year\nBut for now, meri christzamz, everyone! Yea\nMy christzm wish is that\nEach and every one of you\nWill have a christzym\nFilled with love and cheer. Merri christymas\nJoy to the worl\nDoo doo do dooooo, do, Doo Do Dooo Do\nDo do Do do\nJingle bing, jingel bang\nFrosty the snowman\nCom",
                "is_finetuned": true
            },
            {
                "model_name": "grantsl/LyricaLlama",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"All I Want For Christmas\" to be performed by Mariah Carey using the following genres: Christmas.\n\n### Lyrics:\n",
                "responses": "I don't want a lot for Christmas\nThere is just one thing I need\nA crown of thorns and a crucifix\nAll is calm, all is bright\n'Round yon Virgin Mother and Child\nHoly Infant so tender and mild\nSleep in heavenly peace\nAnd we'll be together with the Lord\nIf we obey him\nChristmas time is here\nWith love and cheer\nWe've come to see\nFather Christmas, we have no presents to bring\nBut we won's go until we find him a toy for Tiny Tim\nSo we better find Tinny Tim a Christmas tree\nNow I don\u00e2\u20ac\u2122t know if there' s a Santa Claus\nIt'd be nice to believe in him, I'm gonna keep on believing\nTill I find out, for myself\nThese are the most wonderful words\nThat were ever heard\nThe Christmas season always brings\nBack memories\nOf the happy times\nSpending time with family\nChildren singing carols\nBy the Christmas Tree\nPeace on Earth, goodwill towards men\nEverybody sing\nDeck the halls, fa-la- la- lal-a-lal\nFa-La- La-Lal La La\nHave yourself a merry little Christmas now\nLet your heart be light\nFrom now on, our troubles will be out of sight\nHere we are, as in olden days\nOlden dreams, of the days gone by\nWill be new again\nThis is the time of year\nTo be with those you love\nIs the greatest gift of all\nFor me\nMerry Christmas to you\nOh, oh\n(Mercy, mercy)\nWell, it' a time to remember\nTime to recall\nGather round the family, sing your favorite song\nFeel the spirit of Christmas in your hearts\nSing along\nSilver bells, silver belling\nRing-ting-tong\nDing dong, ding, a ling\nCome on ring\nJoy to the world\nLove has come\nBorn is King\nMay peace abound\nGoodwill to all men of good will\nWonderful Christmas time\nYou' ll find some day\nJust like I did\nHe' lll make your dream come true\nOoh, ooh\nMistletoe and wine\nNodding nearby\nIn the hopeful time,\nWhen the boughs are green\nMy true love will come, come unto me. Come unto, meee\nOn Christmas Day\nYeah\nRejoice, rejoice\nJoin the choir\nCaroling\nAlong the snowy street\nDown the lane\nOutside the window\nShepards bark\nWhy the angel' t\nPlaying their harps\nThey' re singing\nDo you hear what I hear\nListen to them\nPraying for the day to come. To come oh, yeah\nAs we remember the birth of Jesus\nAngels singing, carolling, gloria\nEchoing from the hills\nWhat a beautiful song they're singing now, now now. Now\nWhoa, whoa\nWhite Christmas (White christmas) I\u00e2' d like to snow\nYes I do\nWhile I\nWatch it falling, falling\nLightly\nAround the fire\nKids singing",
                "is_finetuned": true
            }
        ]
    },
    "Blinding Lights": {
        "is_new": false,
        "song": "Blinding Lights",
        "artist": "The Weeknd",
        "genres": [
            "Pop",
            "Synth-Pop"
        ],
        "results": [
            {
                "model_name": "meta-llama/Llama-2-7b-hf",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Blinding Lights\" to be performed by The Weeknd using the following genres: Pop, Synth-Pop.\n\n### Lyrics:\n",
                "responses": "```\n# Blinding lights\n-\nI can't see straight\nThe way that you move\nWhen you're around\nYou've got that healing touch\nAnd I can feel it\nIf you only knew how much I need it, baby\nYeah, you got me feeling like\nBlinded by the lights, yeah\nLike, blinding\nLight, light, lights (Ooh)\nIn the middle of the night, ooh\nDarling, I'm right here\n'Cause I love you so\nOooh, oh\nWe'll be shining bright\nTogether\nNow I know I was blinded\nBut now I see\nIt's blurry, but it'd be nothing\nWithout you here, babe\n(Ooo) You got the love I want\nAll I do is think about you, girl\nSo baby, don' t stop\nJust keep on dancing\nThat' s all I ever ask of you\nOh, it feels so good\nYour love is like the drugs\nDon' tell me it ain' make sense\nNo, no, 'cause you know\nHow it make me feel\nBaby, if you don\u2019t mind\nCould you hold me closer\nPlease don \u2019t let me go\nCoz I feel so high\nHigh, high, so blessed to have you in my life\nTo have and to hold\nHold, hold, to never let go, never\nNever let you go (Nah) (Uh) \nLights, bright lights  (Yuh)  \nSay, say, what you want from me?\nWhat you need from, from  me\nLet' go of your inhibitions\nPut your hands on me  'Cuz I\nCan' see, can 't feel, when you 're not here with me, with, me (Me)   \nMe,  with you (With you) ' Cuz you make it better\nMake it feel like I 'm on top of, on, top\nOf, of  the world\nWorld, the  world, woah\nWoah  woohoo\nMmm, mmm\nHey, hey, (Huh, huh), heyyy\nAh, ah,\nEh, eh\nAy (Ayy) Ayy\nUhhh (Ah) Ah\nShh  shh shhh\nWhoa, whoa\nHeeey\nGirl, shine bright like a diamond\nShow me what I wanna see (Show) Show me everything (Everything) I\u2019m not blind, blind\nRaise your hand, raise your, your\nHand, hand (Hand) And I will hold you down, down\nDown (Down), down (I\u2019ll hold it down) Oh\nMy love, my love (Oh) My love\nThey say that love hurts, well\nMaybe that' 's the reason why\nWhy they always play these sad songs\nOn the radio, radio\nFor you and I, for you I  will go through hell\nAnything for us, us\nEvery day, every day (Damn) Damn\nKeep on pushing, pushing\nPushing, keep it pushing (Pulling) Pull it on back\nBack (Back)",
                "is_finetuned": false
            },
            {
                "model_name": "meta-llama/Llama-2-7b-hf",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Blinding Lights\" to be performed by The Weeknd using the following genres: Pop, Synth-Pop.\n\n### Lyrics:\n",
                "responses": "```\nWe're in the dark, it's hard to tell\nWhich way is up, which way down\nIn my mind, I'm blinded by the lights\nBut I can't tell you why, oh, no\nI'll drive myself insane, insaner\nTo get to where you are, where are\nAnd I, ooh, can only see when I look at you\nYou've got me blind, blind\nBlinded, blinding lights, yeah\nOoh-ooh\nIt'd be a crime if I don'cha\nDon'chu know, know\nThat I want you, want ya\nWant you so bad\nOh, baby, you'da, da\nMake me feel like I got a reason to live\nWhen I see your face, there'sa rush, rushing through my body\nLike a river flood, flooding through me\nSometimes I feel, feel\nSo high, high\nBaby, if you want me, come and get me tonight\nCause I know that you doncha, donchah\nKnow that I wanna, wannah, be with you tonigh\nYeah-e-ah (I can see the light, light)\n(You can bring me to the heights, height) I\nCan'ta wait to see you again\nGot me feeling like a, like, a\nA-a-adrenaline junkie\n'Cuz I just can, just, need, uh, to get a fix\nOn your love, on your, love\nNo, 'cause, cause I need you in my life\nHey, hey, now, whoa, woah!\nWhoa! (Blinds, the blinds) Blinds the,\nThe lights (Oo-oo-hoo) Oo, lights! Blind, y'all\nLight, all the blind lights are blindin' me (Light) (Y'know, they'r\nThey'ren' blind) Y'kno' they, th'y'they', t'th'the'\nAre blindin', blinkin, (Blinkin') Blink, blin' (You know I) You know (K\nAll the) All the (All, al'l' the all,) alll the\nLights (L'ight) L'ights, l'i'ghts (Are blind!) Blinding, bright, night\nNight, nig'ht, ni'gh'ti'h'igh'ts, h'hi'gi'ih'ig, g'high'it'sss\nHigh'its, hi'tis'si'is'iii'ii'ith'ee'he'ye'yes'e'ey'es'a'ai'ay'ah'aha'ha'ho'hu'uh'u'ugh'ug'ugg'ue'o'oo'ou'oh'oa'aa'ae'eh'ea'hh'ie'ei'er'ere'her'here'hy'yi'iy'ish'iss'ess'et'se'te'st'ste'su'so'see'sh'she'sey'ssh'sha'hs'hit'hes'",
                "is_finetuned": false
            },
            {
                "model_name": "meta-llama/Llama-2-7b-hf",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Blinding Lights\" to be performed by The Weeknd using the following genres: Pop, Synth-Pop.\n\n### Lyrics:\n",
                "responses": "```\nI'm in the dark, but I want you to see me. \nWhen I've got no control, I still want to feel something.  \nYou're the only one I see, the way you'll never leave me \ud83d\udcaf\nBlinded by the lights, can't sleep, 'cause I know that I won'\nAnd I feel like I can fly, yeah\nFeel the rush, like a drug, that's inside my blood\n\ud835\udc02\ud802\udd04\ud80c\udcf0\ud804\udc80\ud810\udc04 \u2661 \u1d21\u1d58\u1d9c\u02b3\u142f\u157c\u15e9\u1587\u144e\u14aa\u14da\u1515\u1655 \u029c\u1609\u169c\nIn the middle of the night, when I need a light\n'Cause my love, she don' t love the light, no\nSometimes I don\u2019t know which way to go\nBut I promise that you know, it' s alright\nYeah, you can blame it on the city, blaming it all on my mind\nAll on me, all of my life, oh\nOh, ooh, hey, ah\nAah-ah\nOoh-ooh\nAh, aah  Yeah,\nIt'd be crazy to think that we could ever be  over  yeah,  oh, woah...\nWe could be over\nIf I ever leave, if I leave\nThen it will be forever, forever\nNo, we ain'  t ever gonna be\nOver\nBaby, don  wanna leave  you\nDon  want it, want  it\nWant  to leave you  (I want)\n(Wanna be with you)  Want to\nLeave you (Won' wannabe over)  \ud809\udc26\nCrazy  'bout  your love  and  I  don 't  care\nAbout  the  way  that  we  live  our  life\nGirl, won ' t  never  leave your side\nMmm, mmm\nKnow  what  they  say, they say  \"Hey, boy, what' 's  up?\"\nThey  know  how  bad  my  love is  for  ya\nUh, uh\nHmm  hmm-hmm (Oo) Oh\nTell  me what you want, girl, tell  tell me\nWhat  do  think  about  us?  Do  like  this?\nDo  really  feel  so  good?   Yeah\nThe  lights  are  blinding  but  he  loves  her\nSo  she  won  stay  away  from  him\nHe  knows  his  girl  is so addictive\nShe  can  make  a  man  go  insane\nNobody  else  could  ever  understand\nHow  much  these  two  hearts  hold\nThis  world  was  made  just  because  of  them\nNow  baby, now  now, baby\nEven  though  there  might  be times  when  things  get  hard\nLike  all",
                "is_finetuned": false
            },
            {
                "model_name": "grantsl/LyricaLlama",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Blinding Lights\" to be performed by The Weeknd using the following genres: Pop, Synth-Pop.\n\n### Lyrics:\n",
                "responses": "This is what you wanted\nWhat you waited for\nIt's too late to do nothing\nNo, it'll do you no good\nYou can run away\nBut you know you can't hide\nI can see your future\nAnd it looks so bright\nYeah, I can read your mind\nYour future' s bright, yeah\nBright, bright (Baby, baby, oh)\nBlindin' lights (Yah) , blindin', blind\nOh, you got me like\nWhoa, whoa\n(Burning bright) Blinding lights\nAh, ah, ooh, woo, hoo\nOoh-ooh\nWoo-woo\nLight\nUh, uh\nLights\nNah nah nah, nah\nNa na na\nAahh-ahhh\nFuck it, fuck her, call the lawyers\nDon' t talk to her\nShe' ll tell them everything\nTell them how you broke her heart\nHow you left her in the dark\nGave her nothing to fall back on\nNow you' re on the phone\nSaying that you love her (Love her) (Ooooh) and\nThat you need her back (Need her ba-ba-back) but\nThere' a problem\nThe problem is you don' ve changed\nCause you ain' have no money\nSo you better go and get it\n' Cause she' d rather be alone\nThan be with a broke man\nLike you, like you\nHmm\nWell, well, we could be on our way\nTo a better day\nWe could fly, fly away, away (Away) away from here\nIf you just take a chance\nTake a risk, take my hand\nLet me take you away tonight\nMmm, mmm (Woah-ohh) mhm, mm\nLa-la-love me, love me\nEvery time you call my name\nMy heart just skips a beat (Beat) oh\nCan' be this real?\nCould it be a dream? (Could be) Ooh baby\nFeels like I' m fallin\nInto a hole in my heart, heart (Hole) hole\nIs it too much for you to say\n\" I love you\"?\nDo you even know what love is? Do you? Oh\nSaid, \" I need you\"\nWhy you always say you want me? Say you do\nThen you go away and play with other girls\nDamn, damn\nJust say it once, say, hey\nMaybe I won' give a f-ck\nAbout you and me and all this shit\nAll this bullshit we've been through\nLook, look, all I want is for us to\nBe happy\nEven if that means we have to part\nAt least I know that we were honest\nWhen I look back\nOn what we had\nBack when we was young and reckless\nRemember? Remember? Yeah\nMe and you in a club\nDrinkin, drinkin on that bub\nTryna find the one\nAfter that, after that\nPull up, pull up in that new Benz\nNew Benzo\nGet in, get in\nHead up to the crib\nHave a little sex\nForget",
                "is_finetuned": true
            },
            {
                "model_name": "grantsl/LyricaLlama",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Blinding Lights\" to be performed by The Weeknd using the following genres: Pop, Synth-Pop.\n\n### Lyrics:\n",
                "responses": "I'm going out of my head\nFalling out, falling out\nAnd I don't know what I did\nBut I know that I need it\nMy heart's beating out my chest\nFeeling out and feeling lost\nWhen I've never felt this way\nIt'll be the death of me\nIf you don\u2019t come back\nSo I\u2019m begging you\nPlease don\u2018t leave me out here in the cold\nOut here on my own\n'Cause I can' t take this pain\nNo, no, I, oh\nCan' take it, can\u2019 t, take, it anymore\nOoh, ooh\nBlindin' lights, blinding lights\nThey keep me awake\nOh, yeah, they keep, keep\nMe awak\u0435\nKeep me, awaken\u0435d\nYeah, uh, this feels right\nHit me in my, hit me right in\nThe bl\u0435eding light\nUh\nThis feels, feels\nRight, right, alright\nAlright, ah, ha\nAh, 'cause this, feel, feeling\nNever, never, ever, eve, r\nEver, every, night, lights up\nLights up, up and down\nUp and, down, town\nTown, a, t-town\nYou know, you know\nThat I won' want nobody else\n(Blinds, light)\nLight, (Blins, ligh) t\nHeart, heart\nBeating, bea, thin, g\nBleeding, bleed, ing\nSick, sick, of being broke\nBroke, broke, on the road\nRoad, road, outta town,\nA-a-away, away, ay\nEvery, ev\u0435ry, day\nDay, d\u0430y, in and out the door\nDoor, door, ain' no more\nMore, more, to complain about\nAbout, about, anything\nAnything, any, thing\nThing, y, things\nWon' wanna do\nWithout you, without, with you (Oh) , oo\nLook, look, what you do to me (What you) (Uhh) Ooh (Oooh) Uh-huh (Yah) Ah, yea\nGot me feeling like I lost my mind\nIn the blinds (blinds) and I (I) can (Can) 't\nTake it (Take) anymore (No) uhh\nKeeps me up (Keeping me) up all night\nAll night (All, all) night long (Long, long) oooohh yeah\nCant, cant, candle light (Candlelight) no\nDon' (Don') wan' nobody, nobody (Nobody) else (Else) but you and me tonight (Tonight) alriight\nWe can, we can do what we want\nWhat we, w\u0435, want (Want) toniggt (Ah) haha\nDamn, damn this feelin right (Feel, fe\u0435l, f\u0435el) yeah (Ay) ah\nHow, how, did I get so high (So high) ? (Huh) Whoa (Whoa) yeaaah\nTo",
                "is_finetuned": true
            },
            {
                "model_name": "grantsl/LyricaLlama",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Blinding Lights\" to be performed by The Weeknd using the following genres: Pop, Synth-Pop.\n\n### Lyrics:\n",
                "responses": "You should've known better, yeah\nI'm never satisfied, no\n'Til you put my life on replay, oh\nBut it's alright, it is\nIt'll be alight, aligh-ighlight\nYeah, I can't sleep, so I keep the lights on\nAnd you're the one that keeps me up, up\nBlinded by the light, but I'ma keep dancing all night\nTill we see the sunlight, sun-light-ight\nWon' t you come on over, baby?\nCan' you hear me calling, calling? (Calling)\nWhen I close my eyes, you know I see your face\nLike a magnet, magnet\nSo why you wanna play with me? I wake up to your picture\nThought I was over you, now I don' know what to do\nBaby, won' be over it, over-over-ov-er it\nNo, woah-oh, blinding lights\nOh, let' s go, go-go-g-o\nLet' get, get-get-ge-t it on, on-on\nOoh-woah\n(Let me see you dance, dance) Let me, see, ooh\nWhoa-ah (Let you see me dance- dance dance\nDance dance d-dance) Whoa, whoa\nHey, hey, come over\nJust come and see what you do to me\nIf I could, if I ever let you go\nGirl, that would be the end of me (end of, end) , yeah, ah\nNever, never, ever, e-ever\nAll of my, all of your friends\nThey don wan' see us again\nDon wann' let it go again, again-again\nWe just, we just w-w-won' play the game\nLove you in the morning, love you at night, night-night\nPull me closer, closer\nIn the dark, in my arms, arms\nNow we' re making love in a cold light\nFeel the way you move, move\nWhat you want, what I want\nNothing in this world, nothing in thi world\nCould take you away from me-e-eh\nCome on and dance with, with\nWith me and I won tell you no lie\nYour love is so contagious\nTo me it feels like a drug, drug\nSay you don want me to go (want me) ?\nIs it too much? Is it enough? No\nWhy you won wana play, play\nPlay with my heart? Ooh oh oh, my baby\nMy baby, dan-dan-da-ncing in bl-blinding l-l-ights\nDan-c-cing, danc-ing in b-b-blinging lights, lights-i-ig-hts\nAh, woo, can you feel it? Ah\nThe way that I feel inside? Yeah, mm\nDo you like it when I, when we do it like this? Mm\nAre you ready? Ready, ready, are you? Are you\nReady? Oh, yes, y-y-yes, a-aahh\nAahhh, just come a little closer (come a lil",
                "is_finetuned": true
            }
        ]
    },
    "Lover": {
        "is_new": false,
        "song": "Lover",
        "artist": "Taylor Swift",
        "genres": [
            "Pop"
        ],
        "results": [
            {
                "model_name": "meta-llama/Llama-2-7b-hf",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Lover\" to be performed by Taylor Swift using the following genres: Pop.\n\n### Lyrics:\n",
                "responses": "```\nLove is a game made for two.  \nIt's a little bit crazy and a lot of fun. \nI've got a blank space baby,   and I'll write your name.   \nNow we're as close as we can be.      \nBut we both know I want you close to me.    \nSo here' s to us, baby. Here'ss to always.     \nHere' ss to now.        \nAnd here to forever. You' re my sweetheart.       \nYou' r my best friend.                  .  I love you. I wanna be your man. .\n. Love is our favorite word.          . .       .        .         .     .         \n  Love' is like oxygen. We can't live without it.            \n .                   .             .    .               .              .          \n                                          We' ll make it if we try.           .   ..\n    Love can make you happy.            .      .            ..         ..           ..  ..          ..        ..     .. . ..                         Love. ..   Love  ...  love.. . love .......... .. ..... .. ...   ... . ... ..    ..      ..       .. .... ...    ... .... ....  .... . ...... ....... ....    ....   .... .... .......................  !!!!!!!!\n      I just wan na be with you all the time.              \n          You know that I need you more than anything.             \n        I hope that you don' t mind that i' m holding you tight.               \n            I know you' ve got places to go. But baby i wana go anywhere with. you .!\n         I don ' t want to waste another minute without you by my side.!                \n           I can 't imagine life without your smile. It'd be so hard to live. without. your love. !\n              I could never ask for anything more. Than to wake up every day with your face. right here next to mine. !!\n                I ' d wipe every tear from your eye. And if you ever leave me i ' ll die. ...\n             I would go to the ends of the earth for you my love .!!\n               You ' re the best thing that' v ever seen. and i can promise you that. nothing will ever change. that .!!!  !! !! !!! !!!! !! ! !  :) :)  : ) :) :)))))  ;) ;) :) ;)  ;)) )))))))))))) :))))))) :))) ) ) )))) : ))) :) ) : :)\n!! !   !) )   ) ! : :   :) ! )    :)    : ! ;) !    ;)   ;)    ;    )\n))))\n(((((\n))\n)))\n! ! :)   :\n)\n:)) )!  ) ;\n;)) !)) !! )!! ) !!))!!!)!!))!!!))!))!)!!! )!!!:))))!!))))!!! :)!! :!!:!!!:!!! :!!!:)!!)!!)))!!:)\n(:))(:)))):)))!(:!!):)):):))))):)))!!!;))));!!;)))!)!;!!!!!);!!)!;!:)!!!);)));:",
                "is_finetuned": false
            },
            {
                "model_name": "meta-llama/Llama-2-7b-hf",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Lover\" to be performed by Taylor Swift using the following genres: Pop.\n\n### Lyrics:\n",
                "responses": "```\nLove is like a river\nIt's like an ocean\nAnd you're the captain of my soul\nYou've been calling me baby\nAll night long\nI can't believe it'll ever be you and me\nBut it will be\nCause I'm yours and you are mine\nThis is the love I have been waiting for\nWe'd be together\nEven when we'\nre apart\nSo come on and take me home\nBaby, take my hand\nLet'' s go where the current flows\nTo where it ends\nDon' t let me go\n'Cos I can feel us together in the moonlight\nOoh, ooh\nSweet love is a promise\nThat I intend to keep\nWhen I think of you I\nfeel like I could fly\nThrough the stars\nInto your arms\nFor you\nNothing can come between us\nNo one can break us apart (break us)\nOh, baby, you know that I love you so\nCome on, come and love me like you do\nLike you love the way I do you, too\n`\n````\n",
                "is_finetuned": false
            },
            {
                "model_name": "meta-llama/Llama-2-7b-hf",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Lover\" to be performed by Taylor Swift using the following genres: Pop.\n\n### Lyrics:\n",
                "responses": "```\nLove is a game made for two,\nAnd we're playing it every night. \nThere's a spark between us,  but we can't let it light.  \nWe'll never know if we never try.   \nSo I'm gonna tell you how I feel, I hope you've got the time.    \nI know that you don' know what you got till it' s gone.      \nBut I know I love you, so I won' t let you go.       \nIf you want to love me too,   we gotta go slow.          \nCause I don 't wanna rush into something, that we could regret.         \nYou' re my love and I need you more than ever.        \nEverything has changed, but I can ' t change how much I care.     \nGive me a sign, ' cause I really need to know.                     I wan' na know, wana know wooo, oh, oooh.            \nTell me that your heart is mine, tell me now.           \nBaby I want you to stay, let me know you feel the same.             \nThis love is difficult, can we find a way?            I 'm willing to wait, if you wannna play.              \nWanna know how you ' re feeling, you just have to say.               \nLet me hold you tight, when I kiss you goodnight. I got a feeling that I could die. Oh, my God, it feels like I am flying. When you say my name it sounds so right. But I still don\u2019t know your love as a boy. So I\u2019m giving all my heart, even though I hate to admit that. You\u2019re my only one, whoa, baby. It\u2019s like we\u2019ve been here before. We can make it if I just know we are forever. Baby I, want your, loving. Can I be your lover? Oh baby I. want it all. And I ain\u2019 t afraid to fall. If you\u2019 re willing, to take a chance. Cause you are my, lo- lo lo, love. Lover. Wanna, know? Wannnnnnna knowwwww? I Want you know know love love, l-lover. Love. Yeah, yeah, hey, yeaah, yeeah. Woahhh, woah woa woaaahh. Yeeeeeah baby, aah aaaa. Ooooooh, hoo hhhhooo. Aah ah ah. Hey, ho, ahh, hahahahaha. Hahaa. Haaa, Hoo, Aaa ahhhh ah, aw, aaah haaa haa ha haaaaa. Ho ho ho. Ahh ha.",
                "is_finetuned": false
            },
            {
                "model_name": "grantsl/LyricaLlama",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Lover\" to be performed by Taylor Swift using the following genres: Pop.\n\n### Lyrics:\n",
                "responses": "I knew stardust\nAnd blue jeans\nCould be beautiful together\nSo I wore your high school ring\n'Til the weight of it\nWas too heavy to wear\nNow you're so lovely\nLove has brought me\nThrough the door\nBut we both know\nThis is our first dance\nWe could dance till morning\nYou could take me anywhere\nOh, I could meet you\nAnywhere\nCause I'm yours\nand you are mine\nAll the cards are on the table\nThere's a deep dive into your eyes\nA long and lonely beauty queen\nShe'll be waiting for you tonight\nWhoa, whoa\nwhoa-whoah-oh\nyou've made a believer out of me, oh\noh, you-you-y-yeah\nKnew the devil by name\nbut I knew Saint\nby the way that he loved me (loved me)\nHe'd take my hand and\nlead me through the dark\nThen you came along and rescued me from myself\n(from myself) ,\nyea you did\nso this is what\nlove looks like\nwhen you know you want it all\nWhen you don't want to let go\nwe' ll dance 'til we fall\nBaby, this love is\nworth melting for\nYeah, we could read\neverything that they wrote about us\nSay we' re gonna make it to the end of the road\nIf we keep dancing like we danced that night\nThe first night, yeah\nOoh-oohh-ooohhh\nooo-hooo\nIt' s a love story\nbaby just say yes\nYes, yes,\nyes, baby, just take the reins\ndrive slow\nSinging, \"I could make you happy now\"\n\"I can make this all go away now\nif you close your lips to my heart\ncause you have my love to keep you warm\" (oh) ohh\nMade a rebel outta you and me and that'\nkinda thing\nshouldn' t be so beautiful\nMaybe we were just too young to understand\nThat this was love\nthis was meant to last\nhey, hey\nwoahhhh, woahh ohhh oh oh!\nHey, Hey,Hay\nho, ho,ho\nLa, la,la,lalalala\nlaa lala la lal l la\nlala l l alal alala, ala al al laaaaah! (x2) (yeaaah) Ohh (yeeeaaaeah!)\nSaid you loved her, but you needed me too\nwell I loved you back\nthen I let you in too deep\nnow we gotta do some\ncleaning up to do\nlet' d just leave the past behind\nDarling, i' m just gon' shake\nshake shakin' (wooah oh) shaking (shak sha) it off\nit' aint that deep, it'a just the love you take\ntake it, take it (take) take,take,taketaketake\nt ake, taketakeyekeykey",
                "is_finetuned": true
            },
            {
                "model_name": "grantsl/LyricaLlama",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Lover\" to be performed by Taylor Swift using the following genres: Pop.\n\n### Lyrics:\n",
                "responses": "I knew Saint\nWas a saver\nBefore he saved me\nAnd I've known\nNightmare\nSince the second grade\nBut I never had a lover 'til you\nYou\nBaby, we could leave the Christmas lights up 'till January\nThis is our place, baby, don't go\n'Til the end of the night, stay\nLay with me, spill my secrets\nWe'll build a home\nOut of cobwebs and ros\u00e9\nOh, I want your high, your low, the in-betweens\nYour stormy weather\nJust hold me down\nwhen I fall\nDon' t ever leave\nHold me now\nand I will be brave\nOoh, my heart's been borrowed and yours has been blue\nAll' s fair in love and war\nNow we're sinking, darling, so the loss is all mine\nToo late for you, now I see the rest of my life\nSpent with you painted like a target on my back\nAh, but I love it\nTaylor Swift Lover Lana Del Rey\nAshley McBryde\nPhoebe Bridgers\nJoan Baez\nJack Antonoff\nJessie Williams\nFiona Apple\nMeredith Grey\nKristen Stewart\nSt. Vincent\nDixie Chicks\nSarah McLachlan\nSheryl Crow\nGigi Hadid\nCamila Cabello\nHalsey\nZoe Kravitz\nHaim\nRuby Rose\nEllen DeGeneres\nDebbie Harry\nWinona Ryder\nJanet Mock\nElliot Page\nMark Ronson\nKevin McHale\nLaverne Cox\nAntonio Banderas\nSam Smith\nAlexander Wang\nFrank Ocean\nBrendon Urie\nDan Reynolds\nTyler Glenn\nJohn Cena\nMillie Bobby Brown\nOrlando Bloom\nChika\nJacqueline Novak\nNaomi Osaka\nBarack Obama\nMichelle Obamas Biden\nUzo Aduba\nYara Shahidi\nAmandla Stenberg\nRowan Blanchard\nLevitt\nColleen Hoover\nVictoria Aveyard Jessica Cluess\nMarie Lu\nChristina Lauren\nCasey McQuiston Nina\nAlmada\nBecky Albertalli\nEmily Danforth\nAngie Thomas\nJulie Murphy\nAdam Silvera\nDavid Levithan\nBenjamin Alire Saenz\nAnna-Marie Mclemore\nStephanie Perkins\nMaggie Stiefvater\nNeil Gaiman\nDaniel Handler\nCharlie Jane Anders\nClaire Legrand\nSeanan McGuire\nCarmen Maria Machado\nLeslie Jones\nLucy Hale Emma Stone\nAmerica Ferrera\nConstance Wu\nTracee Ellis Ross\nBusy Philipps\nThandie Newton\nMariska Hargitay\nGeena Davis\nPatricia Arquette\nGreta Gerwig\nDirectors\nReese Witherspoon Oprah Winfrey\nThe Wachowskis\nBig Freedia\nIlana Glazer\nHelena Bonham Carter",
                "is_finetuned": true
            },
            {
                "model_name": "grantsl/LyricaLlama",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Lover\" to be performed by Taylor Swift using the following genres: Pop.\n\n### Lyrics:\n",
                "responses": "I knew stardust\nAnd glitter galore\nWe were both young\nWhen I first saw you\nSo naive\nBut I fell hard\nYeah, I was there\nYour chest was open\nHeart was beating\nMine was too\nYou said: \"Take a chance\nTake the fall\"\nThen we lost\nAll of our minds\nIt's gorgeous\nBeing in love\nIs a chemical\nReaction\nBaby, can you feel\nMy heart beats for you?\nIn the middle of the night\nWondering if you know\nThat I know the colors of your eyes\nThe taste of summer in your lips\n'Cause we've got a love made of\nLoves that last forever\nThis is our song\nClosing credits\nRunning through the streets\nWith you, my love,\nOh, you're the one that I want\nOoh, ooh\n(Oh)\nLooking at you in the bathroom mirror\nStanding there in just your underwear\nSaying: I love you to myself\nDo you ever think about us? (About us) ?\nNo, no, we can't forget\nAbout the times we had\nBack when we were younger\nThose nights were amazing\nHey, hey, don' t you remember? ? (Remember?) ?, oh\nHow you held me\nNever letting go\nTill the sun came up\nOn the street we both live on\nAin' ty, ty for the good times\nBad times, they'll come and go, whoah\nWhoah-oh-whoah Whoah ooohh\nNow we got the world at our feet\nEveryone is watching us\nFifteen lights on the boulevard\nWatching time stand still\nSwimming in a pool of champagne\nGot my bags and I'm out the door\nGuess it was a business\nNot a pleasure\nTo be with you so long\nDidn' y know? I didn' te think\nThey'd let me stay forever? Whoa\nSaid: Take a bow\nHere' s a flower\nFor your button nose\nJust because\nDarling, it' ll always be\nSummertime\nHolding hands in\nRestaurant gardens\nDancing in my kitchen\nDon'te know why\nThere' d be no sunshine\nIf you weren' le here\nCome on, come on!\nLet' go! Whoo! Oh-o-oooh! Oooo\nCan you believe\nWhat we used to say? \"Oh\"-e-y\n\"We' re gonna be the greatest\nShow on Earth\" (On Earth) , oh-woahh! Woah! (Woah!)\nLa-la-lalalala\nLo-lo-low-er\nAh, ah, woah (Whoa!) , ah (Ah!) Oh\nMe and you are like\nTwo ships passing in th\u0435 night, yeah\nStill, somehow\nSomehow, someway\nSeems like we always end up together\nLike a lightning bolt out of nowhere\nEven when I fall\nBoy you always take me back in (",
                "is_finetuned": true
            }
        ]
    },
    "Electric Touch": {
        "is_new": true,
        "song": "Electric Touch",
        "artist": "Taylor Swift and Fall Out Boy",
        "genres": [
            "Pop",
            "Pop/Rock"
        ],
        "results": [
            {
                "model_name": "meta-llama/Llama-2-7b-hf",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Electric Touch\" to be performed by Taylor Swift and Fall Out Boy using the following genres: Pop, Pop/Rock.\n\n### Lyrics:\n",
                "responses": "```\nI've been trying to get you out of my head\nIt's been a long time, I can't let you go\nYou're like a drug, electric touch\nThe more I'm around you, the more that I want\nSo take me to the edge\nAnd keep me there, keep my heart open\nWhen I get close to you\n'Cause I know I could fall\nFor the electric, electrifying touch of you \n  - Taylor\n    - Swift\n      - Fall\n        - Out\n          - Boy\n            - Electric\n              - Touch\n                - Me\n                  to\n               - the\n             - edge ",
                "is_finetuned": false
            },
            {
                "model_name": "meta-llama/Llama-2-7b-hf",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Electric Touch\" to be performed by Taylor Swift and Fall Out Boy using the following genres: Pop, Pop/Rock.\n\n### Lyrics:\n",
                "responses": "```\nThis is a lyrcis I wrote for the song \"electric touch\" by taylor swift and fall out boy. I hope you enjoy it!\nI've been waiting for you\nYou're the only one I'm thinking of\nAnd I can't wait to hold you tight\nOh, I want you to feel my electric touch\n(ELECTRIC TOUCH)\nSo baby, just come to me\nWe can make this electric\nBaby, you can feel it too\nMy heart is racing, my breath is taken away\nWhen I see you, it's like you'll never leave\nCause I know that you feel this way\nLike I do, like I feel\nDon' t be afraid to let it out\nLet it all go, let yourself go\nJust let me hold ya, oh, baby\nEven if it hurts\nIt'd be alright, cause I would never hurt you baby. (ELECETRTOUCK) (electrtoouck) \nYeah, yeah, we can do this\nTogether, together\nOoh, ooh\nGonna make you fall in love\nMake you want to stay\nBut it won' be easy\nWon' let you go away, no\nNo, won let go of you (oh) no no,  (no no) oh\nCome on, come on\nFeel my heart, feel me now\nFall in to love, fall into love  with me and you.  Yeah\nBe my love and I will be your love too, babe\nAll I need is you and me, so come and let' s make love to you  yeah\nLove to, love you too  baby  I love love loving you love yeah  ohh\nHold on to my hand, hold on tight, (hold on) hold tight (yeah) yeah. Hold tight  love. Yeah. Ohh. Ooh. Fall in, into  into, in  in. Into love with you yeah yeah (fall into) into you with love (into love) with, with (with love). With, wiith love wiiith lovve. Wiiiiiiiith looooooove  wiiiit looovve  loove. With love baby yeah with lo. Love. Baby. You love me too. yeah love in with. wihth loovvve with lovvvvee. with lova with  lloooove with woo with with\nlove baby with yooouuu. oooohhhhhh yeahh (wahhh) ooohh wahhahahaha. Haha\nooowwwwww. hahaha yeahhh\noowwww (oo) waaaaaah (waaaa) haaaah. Haa. Hey. Yo. Whoo. Whoa  whoa whoaaaa. Woah whooo whoo\nwhoooa ooww. Waah waa wow. Ahhh. Aah haa haah  ha. Hee. Ho. Hu. Uuuhhh (uh) uhh  uh uhuhh huu\nuhuu  hu. uuuaa (uah). Uah uah huuh uuhh huhu. huh hu hu hh hhuhu hu u u\nhu u hu  hhh h h",
                "is_finetuned": false
            },
            {
                "model_name": "meta-llama/Llama-2-7b-hf",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Electric Touch\" to be performed by Taylor Swift and Fall Out Boy using the following genres: Pop, Pop/Rock.\n\n### Lyrics:\n",
                "responses": "```\nI'm a lightning rod, I'll never be your girl\nBut I want to feel electric touch\nYou're a firework, you've got to let it go\nAnd I know you get the spark, but you won't let me in\nEven when I open up, it's like you just can'\nt see me\nSo I take a deep breath and I say, \"I don' t need you\n\"\nThe last time I saw you, the last thing you said was,\n\u201cI just want you to know, that I love you so\nIt' s like I can\u2019t breathe when you\u2019re not near\nLike I\u2019m drowning in a sea of tears\nNow you can brea\nthe silence, don\u2019 t say it\nDon\u2019' want it, oh no\nLet\u2019s leave the past in the dust, let\u2019\u2019 go, go (go,go)\nJust leave, leave it all behind\nIf you want me back, then you know where to find me (find me)\u201d\nNo, no, not me, (not me), no (no) (me) I won\u2019\n\u2019 (I won,t) be (be) your (your) girl, girl (girl) \n(gir\nl)I\u2019ll (\u2019I) never (never) let (let) you (you) in (in) again (again) \u2019cause I\n' m (m) a (a) light\nning (lightning) rod\nNever (Nev\ner) gonna be yo\nur (yo\nu) gir\nll, gir (gi\nrl)\u2026\n\u2026(ne\nver, nev\never, never, ever, ne\nv\ne\nr, n\nev, ev, e\nrv, erv,  e, v,v)r\nR\nO\nD\nL\nY\nU\nS\nT\nH\nA\nC\nK\nW\nHO\nRE\nV\nER\nB\nF\nOR\nG\nIV\nEN\n, B\no\ns\ni\nn\ng\nm\ny\nh\na, h\np\nw\nho\nre\nve\nrs\nb\noo\nd\nf\nor\ngi, gi\nvn, gn, o, r, s, th, t, y, u, f, l, a, b, d, c, k, w, or, for, giv, en, me\u2026\u201d  (giv\nen\nme, m\nee,)me\nME\nM\nEE\nMM\nEM\nSE\nMY\nTH, T, H, O, R, E, V, ER, S, F, OR, G, IV, EN, M, Y, HO, RE,V,ER,S,B,O,SI,N,G,MY,H,A,HP,WHOREVERS\u2026. (W, W, wh\norever\nvs\n.vs.v.s.o.ver.e.r.se.m.y.h.a.p.w.ho.re.vers.b.oo.d.f.or.gi.vn.g.n. o. r. s. t. h. a. c. w. ho. re",
                "is_finetuned": false
            },
            {
                "model_name": "grantsl/LyricaLlama",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Electric Touch\" to be performed by Taylor Swift and Fall Out Boy using the following genres: Pop, Pop/Rock.\n\n### Lyrics:\n",
                "responses": "I've got a feeling that it's time to let go\nAnd I'm holding on, I can't let you know\nYou're the only one, you'll be the one\nTo make me feel this way\nThis way, this feeling\nIs getting stronger\nAs the days go by\nIt'd be so easy to just let it go but\nThe harder I try to resist it\nJust won' t do\nWhen you touch me like this\nLike this, like electric touch\nEven when you say that you don' s need it,\nBut I know you do, do you\nBaby, don t you let me go, go now\n'Cause I feel it coming on\nDon''T you feel the electric\nFeel the touch, the way I do?\nYeah, it is true, that I wanna be with you, oh\nWell, what can I say? I never meant to\nMake you stay, but I don 't want you to go. Oh\nSo baby, baby\nPlease don\u2019t you go away, no\nOh, please don \u2019t go yeah\nNo, not tonight\nNot tonigh\nC'mon, c' mon, come on baby yeah, yeah yeah. yeahh\nCome on yeah come, on c\u2019mon baby. come\nOn\nDo you really think that\nThat I would let this go? No way. no wayh. ohh, yeaahh!\n(Ehhh\u2026)\nNow, now, can you see, how it\u2019s meant for you and me? yeahhh\nLet\u2019 go of me, let\u2019 me be, just for tonigth\nWe\u2019re just gonna be\nAll right, all right\nAw, aw, aww, wohhohhooh\nWoh, ooh oooh. oooohhh. yeaaahhh! ohhh, woah oh, ahhhhh oh. ahh oh oh ah oh! ah! yeah! yeaaaaah! wow! comeon! cmon! owooowooo! aah\nAhhh yeah yehh yeh yeah yehaa\nWhoooooahahhaahaha\nOooouuu, whoooaohohoo. hooaaa. aaaooow\nHoo, hoho, hooha! hahaaa! woohah haa, haahaa\nHaah aaaaaaa. haaa, heeeeee. hey heyyy\nHeeeyyy\nUhh oow, uhh ah, aaah ah\nWhah ohooo\nMmhhmmmmm\nGoo hhoo\nGooh ho\nRrraaaawwwr\nDoo doo dooo! do do. dooh doh dohh doho doah doa doaa dooa do ho do oh ho ho ohho hooo ho,ho,oh ohoh! ho.ho\noh yeah oh yeah owhhhhoa oh owwwwhoahho ahhoho! hhaoo haoo ahah hhha hahah. woo ohaoh ha oh ha ha hoah ho ah haho o ha o hoa ho ha\nha ho aho ha,hho hhhha oh h o o h a o oh a ha. ho! whooaa oa oahooho",
                "is_finetuned": true
            },
            {
                "model_name": "grantsl/LyricaLlama",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Electric Touch\" to be performed by Taylor Swift and Fall Out Boy using the following genres: Pop, Pop/Rock.\n\n### Lyrics:\n",
                "responses": "She was a girl, I was the boy\nWe were young and we were wild\nI was 17 and she was so beautiful\nThe lights were out and the music was loud\nAnd we danced all night to a rock 'n' roll song\nOh, it was magic\nBut it's just like everything else\nIt was gone too soon\nSo I'll remember\nWhen I look back on that night\nThat I felt electric\nEven though it wasn't right\nHer eyes were closed and her head was back\nLike a supernova she exploded\nThen she disappeared\nInto the night like a shooting star\nDidn\u2019t even leave a spark\nNo, she didn' t even dare\nTo leave her mark\nNow I know\nWherever she is\nWhatever she' s doing\nWhoever she\u2019s with\nHe' ll never know the way\nYou make me feel\nNever know, never, know ever\nKnow, ever, no, ooh\nOoh, electric touch\nMy heart is racing and my head is spinning\nFeel the rush of blood to my skin\nYour electric, your electric kiss\nIs the only thing I miss\nAin' tonight, we're still young\nTomorrow we might be gone\nTonight we dance, tonite we sing\nLet' the moment last forever\nHold on, hold on\nBefore it all goes away\nJust hold me close and never let me go\n' Cause I need you to\nMake me believe in love again\nGive me your hand and I will give you my heart\nBaby, baby, give me that electric love\nDon' touch me, don' look at me\nSay you' re not ready for this\nPut your hands up and feel the sparks\nCome alive, come alive\nGet a little bit closer\nShut your eyes and let your heart go, go go (oh)\nGo, oh, yeah\n(I can' feel it) I can feel electricity\nAll over my body\nCan you feel me?\nTouch me and take me away tonigh\nTake me tonigth\nYeah, touch, take, feel, away, night, all, me (yeah) , you\nFall, fall, in, love, with, you, again, my, friend,\nLove, is, not, just, a, game, to, play\nBe, brave, open, up, be, honest, talk, about, the, things\nMaking, us, sad\nStay, stay, true, keep, going, on and on (and on) and\nOn (on and) on , on on .\nThis is not a love song and it\u2019 s not\nAbout you and me but it is about us\nEveryone who believes in the power\nOf love and everyone who\u2019 ve ever been\nLeft out in cold\nOr left behind\nAnyone, anyone who feels\nThey' ve been pushed aside\nIf you believe you can make a difference\nStand up for yourself and be heard\nCause love is powerful, beautiful, strong\nDangerous, hard, real, worth fighting for\nOne day, one day we\u2019 ll all be free\nFree to love who we want to free to live\nAs we wish, as we please\nUntil that day\nKeep your head up high, hands on your hips\nLook them straight in",
                "is_finetuned": true
            },
            {
                "model_name": "grantsl/LyricaLlama",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Electric Touch\" to be performed by Taylor Swift and Fall Out Boy using the following genres: Pop, Pop/Rock.\n\n### Lyrics:\n",
                "responses": "I'm a lonely girl\nWith a broken heart\nBut I'll never show it\n'Cause I don't know how\nWhen you look at me\nYou see a pretty girl with pretty eyes\nAnd I hope that you never realize\nThat I never know why\nWhy I can's cry\nMy eyes are dry\nDry as the desert sky\nAll I need is your electric touch\nYour electric, electric\nEver touch, ever touch to make me cry, cry cry-eyes\n(Eyes, eyes, dry eyes)\nOh, I could be your girlfriend\nIf you could just see\nBeyond the pretty face\nTo the loneliest girl you've ever seen\nWho'd never leave you\nNever hurt you, no\nShe'ld just need you to touch her\nJust touch me once\nSo I know that I am not alone\nCry, my eyes cry dry as desert skies\nEvery time you touch my face,\nIt starts to rain\nRain, rain, like the heavens crying\nLike the skys are cryin'\nThen my heart will start to beat again\nStart to feel again, again again-again\nOoh, ooh\nLet me feel your touch again (touch again) (your touch) again. (agian) Oh\nYeah, yeah\nTouch my lips again and I will cry (I will) I, will I cry again? (oh) oh\nTell me why, why you can touch a girl and make her cry?\nMake her eyes dry? dry like desert\nsky? sky? cry like\nskies cry when it rains? ra-ains? yeah-yeah-oh-hey\ncry like skyes cry rain like rai-rain like rain-like rain (rain) yeah (yea-e-ah) ooohh\ntouch me touch (me) me me, me oh, oh ohh ohhh oh (ooohhhh) ahhhhh\nyeaaahhh ahhahahhahaahhaaah hahhaaaa\naaawwww awwww\nooooo ooo o o oh o-o-oo-ee-aw-aaw\noawww oaww o aw o a oowww a oh a aah oa oah\nah ah ah\noh oh ah oh ha ahahaha\nhee hee\na aaa a aw aaw a aaah a ah a\naw aaaw o ah o\naaaaaaaaa ha haha haahaa\neeeee eee ee o e e oe ah e ah ha e\nh eh heh haa ah h eah eha ehehehhehhheeeheeh\nhaaaaaaa haaa ahaha ahaah aahe haaaaaahaaaa\naha ahha ah aa aaaaa ha a ha\nhhha ahe aeheaheaeheaaheahaeh aehhehaeaeahhe\naeaaeaeeaaaeaaeeaa ea e a eae aeeaeeea hhaheahahe ahhe hhe hehe eeh e h hh h ha h\ne a he e ha he hahe aa e hea he a h aaha ehh aae eaa heaa hhh hae haeeh ah he\nehh aa haeh aeihe",
                "is_finetuned": true
            }
        ]
    },
    "Next Thing You Know": {
        "is_new": true,
        "song": "Next Thing You Know",
        "artist": "Jordan Davis",
        "genres": [
            "Country"
        ],
        "results": [
            {
                "model_name": "meta-llama/Llama-2-7b-hf",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Next Thing You Know\" to be performed by Jordan Davis using the following genres: Country.\n\n### Lyrics:\n",
                "responses": "```\nI'm not gonna sit here and say I've got it all figured out\nBut I know that you're the one that I wanna get to know\nAnd I can't wait to find out where this goes\nSo I guess I better ask you out on a date\nWe'll take it slow and see where it goes, yeah\nYou'd be the first to hold my hand\nNext thing you know, I could be saying \"I do\"\nWhat are we waiting for?\nLet's go, let'\ns do\nTake it easy, we'l\nl be fine\nJust you and me\nGirl, you look so good\nDon'' t know what I did to deserve you\nIt' s like I fell in love and I just can\nt get enough\nIf I was gon\nna tell you what'sa on my mind\nWould you come\nout with me tonight? I don' know where we\nll go\n'Cause I gotta\nhold you, baby\nHold you tonigh\nht\nNow I want you to\nknow I ain\ngot a lot to show\nBaby, there' a\nlot of things I\ndon' wan\nta do without\nOh, girl, if\nyou' re gonn\na make me fall\nin love\nThen I hope you w\non' th\ne same page\nAs I am\nCuz\nif I get you alone\nTonight, then\nwe' l\nbe fine,\nyeah\nThere' ll be no\nsecond guessin\ng\nCan' be\nno second\nguessing\nWhen I look\ninto your eyes\nYeah, it' sa\nn easy\nthing to do, but\noh, lord\nIs it\never\nm\nakin' me wann\nask you ou\nto be my girl\nWell, now I wa\nnt you t\no know I aint got a l ot to sh\now\nbaby\nHey, hey, come here\nLook at you now\nStanding in my\ndoor\nway\nCome on in, babe\nMake yourself at home\nSit down, relax, kick off your shoes\n(Oh) I been waiting all day for you (Oh yeah)\nTo come on over\nShow me what you got\nShake it, shake\nit, like you mean it (Like you) Mean it like\nlike you do (Yea\nh) Yeah, oh, no, don\nget it twisted\nKnow you don\u2019t have to prove it to me, love (No) But\nlet me see what\nelse you\u2019ve\nAin\u2019 t got no time to waste\nEvery second I\u2019m here I, want you right here with\nme\nRight here, right he\nre with m\nee\nHee\nhee, a-ha\nha, ha, yea\nahh\nAh, ah, ooh\nOoh-hoo\nho, ho, whoa, woah-ohh, hmm\nmmm, mmm, mm\nMm-mm-\nhhh-hmmmm (Hmm) Mmm h\noooo-oooohhh\nUh uh huh\nuh-huh (Uhuhh) Uh oh",
                "is_finetuned": false
            },
            {
                "model_name": "meta-llama/Llama-2-7b-hf",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Next Thing You Know\" to be performed by Jordan Davis using the following genres: Country.\n\n### Lyrics:\n",
                "responses": "I'm just tryin' to get by\nAnd I'll be damned if I don't\nNext thing you know, I got a baby on the way\nGot a new car in the driveway\nAin\u2019t got no money in my account\nBut I ain' t worried 'cause\nThis ain\u2019 t nothin' new\nWe've been here before\nJust a little bit older\nSame damn thing\nIf I had to do it all over again\nWould I change a thing?\nNah, not one damnnn thiiing\n'Cause I love you\nYeah I do\nBaby, you're everything\nSo I guess I won'\nCuz I know I lost\nThe next thing I knew, we had a family\nFell in love with a kid I never knew\nLife is good, ain t no doubt\nIt's a dammmn good life\nWhen you got someone you love\nLove is a beautiful thing, yeah\nThat'd be the next\nThing you knew...\n",
                "is_finetuned": false
            },
            {
                "model_name": "meta-llama/Llama-2-7b-hf",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Next Thing You Know\" to be performed by Jordan Davis using the following genres: Country.\n\n### Lyrics:\n",
                "responses": "```\n\"Next thing you know, I'm in love with you,\nAnd I don't know what to do, \n'Cause you're the only one who's ever made me feel like this, oh\nIt'd be so easy to say I love you too, but\nNext time I see you I might be crying, yeah\nI don\u2019t wanna fall in like with somebody I can\u2019\nt get enough of, no\nSo I\u2019m gonna try to play it cool, pretend like I just don\n\u2019 t know you\nBut you can tell by the way I look at you that I\ncan\u2019\u2019' t help myself\nYeah, next thing I know I\u201dm falling for you\"\n- Jordan davis\n````\n",
                "is_finetuned": false
            },
            {
                "model_name": "grantsl/LyricaLlama",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Next Thing You Know\" to be performed by Jordan Davis using the following genres: Country.\n\n### Lyrics:\n",
                "responses": "Yeah, you got me spinnin'\nAnd I can't stop\nYou got my mind up in the clouds\nI'm thinkin', \"I don'T know what I'd do without you\"\nIt's like the more you show me\nThe more I wanna know\nCause I don\u2019t wannna let you go\nGirl, I could see myself lovin\u2019 you\nFor the rest of my life\nIf you're ready, then I am too\nBaby, let'S go all in\nOh, baby, all I know is that I love you, girl\nBut I ain'Lt got a clue what comes next\nSo I guess we'll just keep on loving\nLoving and lovining\nWhoa, whoa\nWe'Re just gonna keep lovving\nNext thing you know, we could be\nSittin on a porch swing\nHolding hands, sippin wine\nWatchin the sunset go by\nWith the kids playing in our yard\nTalkin bout the good old days\nWhen we met and fell in love\nWell, that'D be a pretty good life, huh?\nThat'Da the next thing I see\nMe and you sittin by the fire\nDrinkin a little wine,\nWalking on the beach\nGoin back to our room\nMaking love all night\nIn the morning, waking up\nLooking at each other like\n\"Babe, what the hell did we do? \"\nHow've we made it this far\nWhere we are today? We got it good\nLet'Me say it again, oh\nAll I knoow is I lovve you girl, but I aint got no clue\nWhat comes nexxt\nso I gues we just gotta keep goin\nNexxt thing we know we gon be sittn on that porcch\nSwing, holdin hands sipping wine watchin sun set go, by with the kid playing\nin our yard, talkin about the go old day when we me and we fell\ninto love, well that be pretty dang good, yeah\nthat be the nect thing that i see me an you sit by fire, drinkin little\nwine, walkin in beach, go back room makin love at night, in mornin wakin up lookin at\neach other, like babe what did, did do, how we make it, this, far, where we\nare today, got good let me say again oh all i know i love girl but i a,nt got clue, no\nclue whats next so i guess, i just go on, next, thing, know well we go be sitting on\nporch, swing, holding hands sipin, wine watching sun, set, g,o by, with kidd, playin yard, talkin\nabout good ol days when, me, an we, fell, into love well, thats, be, pretty, darn good yeah, tha be next, think, knwo, sit, fire drink wine walk in, beach go in back, room making love night in morning wake up looking at, each, other bae what we did did how, made, it far where, are, today got, good ne,xt think know me you gona",
                "is_finetuned": true
            },
            {
                "model_name": "grantsl/LyricaLlama",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Next Thing You Know\" to be performed by Jordan Davis using the following genres: Country.\n\n### Lyrics:\n",
                "responses": "I'm on the road again, I've got a long way to go\nAnd I can't wait to see you, but I don' know when\nYou're on my mind all the time, girl, that's the truth\nBut I guess I just miss you\nWhen I ain' t around\nThe next thing you know\nGirl, you'll be calling me\nSaying you wanna come back home\nTo the next, next\nThing, thing, yeah\nYeah, the thing that you knew\nOh, oh, baby\nThat' s the way it' ll be\nWon' be long\nBefore you say you love me again\nDon' wan' to have to leave you again but\nIf I do, if I gotta leave\nIt' l be the last thing I wann' do\nCause I know, know that I love you and\nWell, there' something ' bout you baby, something that\nMakes me want to stay\nSo I hope you don', don ' miss me too much\n'Ceause the, 'cause you ain ' t gonna be alone\nFor too long, no\nBaby, cause the\nNext, yes, it, well\nJust like the first time\nLike you always said\nWe' re gon ' be together\nAs long as we both shall live\nTill the end of the world\nNo matter what, we' ve got forever\nUntil the day that we die\nThere ain, ain no need to worry\nAbout me and you parting ways\nNot as long we got each other\nIn our hearts\nLord, Lord, let it be that way\nLet it stay that, stay, way, hey,\nHey, alright\nOoh, ooh\nBe that thing\nKnow you ' ll love again and again yeah, come on\nCome on, a little bit\nMore\nA little, little more\nAlright, just like I knew it would be, be alriight\nHow, how, when, where, why\nAll of it\nForever, for always, always and forever, forever and ever\nCan' keep you from me, me from you girl\nNever, never be apart\nOnly thing in my heart\nIs you in me girl and me in you yeah yeah oh oh\nLooking out for number one, two, three\nOne, one two three, four\nThree, five, six, seven\nEight, nine, ten\nWhoa, whoa\nHere we go again hee, ha, hoo\nHee ha hoho, ho, here we are\nBack in the game again so\nWhat I, what I gonn' say to you now\nAfter all this time away\nSaid, said, all I want, want is you back here\nWith me baby and baby I\nMiss you more and more every day\nEvery day, every night\nFeels like\nThis might be right\nFinally, finally, this is right, right for us\nRight for you right now, now ohh\nBoy I need you here right here with me now yeahh, yeaahh ohhhh yeahhh\nNow, take me home, home to where I belong\nWhere I was born and raised\nTake me back, back to the place\nWhich I will always love\nHome,",
                "is_finetuned": true
            },
            {
                "model_name": "grantsl/LyricaLlama",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Next Thing You Know\" to be performed by Jordan Davis using the following genres: Country.\n\n### Lyrics:\n",
                "responses": "You know that I've been in love with you since the day I saw you\nBut I never told you 'cause I was too afraid\nAnd I know it's time for me to leave\nI'm walking out that door\nSo I gotta tell you before I go\nBefore I leave this place\nThat I love you more than I could ever say\nMore than words could say, more words than you could know\nBaby, you're my world, I don't wanna lose you now\nOh, the next thing you know, we'll be walking down the aisle\nYeah, yeah, oh\nThe next time you see me, well, it won' t be alone\nIt' ll be with my beautiful bride\n'Cause the very next, next\nThing you, thing that you do\nIs gonna be walk me down that aisl\nOoh, ooh\nSay the words that make me yours\nMake me your husband\nTell me you love me forever and ever\n(Love me)\nAfter all this time, baby, come on\nLet me hear you say you wan' me\nTo be your man, your only man\nYour everything\nWon' be long 'til I see you again\nUntil then, just know I think about you every day\nEvery night, every second of the night\nWhen I close my eyes, girl, when I kiss your lips\nMy whole world' s right there\nIn your arms, in your eyes\nJust one more time\nGirl, before you go, tell me that\nThis is the one thing I wann' know (Know) , know , oh, know oh oh yeah\nKiss me one last time and tell m\u0435\n\" I do, \" I, do I\nDo, say the word, babe\nWe' re gon ' be married\nAll of my life\nNow, this is it\nThere' 's no turning back\nDon'T you ever wana go back to the days\nWhere you were all alone?\nNo, no, never, ever, ah\nIf you never let me go now, then\nWhy don ' t you tell ' em\nNext thing, that next thi, ng that thing\nWhat you gonn' do? You gottn' say \"I do\"\nThen we gettn ' married, married down\nAisln'\nDontcha know? (Downtown) Downton\nDown the isle, down, is, le, hey\nHey, alright, woh yeah yeah alri, ght, y' all, all\nCome on, let' S get married now (Now) now now yeah now ohh\nOne more kiss, one m\u043ere kiss\nFor the road, for the roa, hh ohhh\nAlright\nWell, now that we,\nwe' ve said our vows\nand we are, are officially married (Married) married oh now ooohh yeahh, eyeyeeyeyyeah\nLook, look, at what we did\nCongratulations, man (C'mon) c' mon, cmon\nLadies and gentlemen\nPlease welcome my wife\nShe is my everything, my one and only, Mrs. Davis\nMrs. Mrs, Davis, Mr. Mr",
                "is_finetuned": true
            }
        ]
    },
    "Penthouse": {
        "is_new": true,
        "song": "Penthouse",
        "artist": "Kelsea Ballerini",
        "genres": [
            "Pop",
            "Country/Pop"
        ],
        "results": [
            {
                "model_name": "meta-llama/Llama-2-7b-hf",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Penthouse\" to be performed by Kelsea Ballerini using the following genres: Pop, Country/Pop.\n\n### Lyrics:\n",
                "responses": "```\nPouring rain outside\nYou're inside with me\nAnd I don't know what to do\nI don\u2019t wanna be alone\nWith just me and my thoughts\nSo I'll take you home\nTo my penthouse\nA place where you've never been before\nJust you and me in my bedroom\nWe'd be in the middle of nowhere\nBut I know that you wouldn' t mind\nIf I told you\nThat you are mine\n'Cause I want you to stay\nAll night long in your pent house\nClimb that staircase\nUp to my room\nLet's get lost in this pent-house tonight\nSay that I can keep you here forever\nBaby, I could stay with you all night\nIn your room in our pent home, pent pent\nYour pent, my home in a pent up pentagon\nIt'sa pent room, you can call it your own\nYeah, it'a pent roof, we can be on top\nHangin' out in pent houses\nGot a lot of pent rooms\nThey'r pent in\nOh, oh, yeah\nTell me that we'l be together\nFor the rest of our lives\nNever have to say goodbye\nWhen we make love in that pent town\nWhere the stars are like diamonds\nOn the ceiling\nThe room is like heaven\nLike a fairytale\nLove is in every corner\nOf this room that'st ours\nEverything' we do here\nIs our little secret\nNo one else is invited\nThis is our love' pent place\nOur pent love\nSafe and sound in love pented up\nStay forever in one penton\nMy pent penned pent down pentacled pental\nWriting in pencil pent on the pentium\nDedicated to the pen pent Pentagon pentant\nWrite in pen Penthouse pentatech pentameter\nEverlasting pentent Pentecost penteco pentet penting\nPut it in writing Pentium pentagram pentagonal pentad\nPeople pentatonic pentacle pentathlon\nMelodic pentaton Pentas pentapod pentangle pentanome pentane pentas\nFive-sided pentahedron pentamer pentastich pentaspid pentavalent pentax pentine pentil Pentland\nMake it last forever pentland Penton Pentol Pentomino pentomic pentom\nSpend the night in Pentel Pentex pentile pentipent\nBeautiful pentin Pentone Pentothal Pentra Pentu Pentyl Pentyn Pentzer Pentz\nProtect your Pentum Pentus Pentwater Pentworth Pentwood Pentyr Pentypent Pentalogy Pentangle Pentanet Pentapus\nLong-last Pentazen Pentax Pentacanth Pentad Pentacle Pentagram Pentagons\nRepeat Pentam Pentate Pentat Pentath Pentav Pentaxis Penthe Penthel Penthi Pentina Pentil\nHeaven Pent Pentian Pentinum Penta Pentite Pentit Pentode Pentodon Pentep Pentet\nRhythmic pentode pentel pentop Pentoplas Pentoph Pentopt Pentor Pentos Pentox Pentry\nDo you want to Penturn Pentw Penty Pentzen Pentoz Pentzo Pentza Pentzi\nGo",
                "is_finetuned": false
            },
            {
                "model_name": "meta-llama/Llama-2-7b-hf",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Penthouse\" to be performed by Kelsea Ballerini using the following genres: Pop, Country/Pop.\n\n### Lyrics:\n",
                "responses": "```\nYou are the one for me,\nThe one that I adore. \nAnd when I'm with you,  I feel like I can be anyone.  \nAnything. I am a queen. A queen in a penthouse.    \nA queen, in the pent house.   \nI'll never let you go.     \nNo, no, you're my home.        \nMy home, my sweet home....\n                          \nPull me close, I want you to know,    I wanna be close to you.      \nTo you....      I don't know what I would do without you...\nWithout you\nOh, oh, ooh, yeah\nIn my pent-house, we've got it all.           \nWe'd never have to leave.           No, not ever.       \nEverything is perfect. We'r perfect, baby.         \nPerfect, perfect\nOooh\nSo, so, sweet\nThis is the place I call home\nHome, home    This is my place\n(Ooh)\nIt's the only place that'\ns ever felt like home to me\nHmmmm\nWhen I met you I was in such a dark place.            \nBut you showed me light.             \nYour love is all that matters to\nme.                                      Me. Me, me.              \nMe. You showed the world what love can do.               \nWhat love could do to save me from the dark. My dark\ndark. Dark. (Oh)  Dark\nDark\n  Oh, Oh\nNever let go, never ever let me go\nLet me be, let be be with\nyou\nBaby, don\u2019t ever leave me alone\nDon\u2019 be gone, just stay with me and\nbe mine, be mine\nBe mine. Be mine\u2026\nNow that we\u2019ve found each other\nThere\u2019s no one else I\u2019d rather be\nThan with the love of my life\nBy my side. By my\nside. Side. Oh!\nSide\nTogether we can make it through\nAnother day. Another day\nDay, day, another day (Oooo)     Another\nday\nOne day at a time, one day a\ntime, One day time. One, two, three\nFour. Four. Five, six, seven, eight. Nine. Ten. Eleven, twelve. Thirteen, fourteen. Fifteen\nSixteen seventeen, eighteen nineteen, twenty. Twenty one, twent two. Two two two\ntwo two Two, Two. Three, Four, Five. Six, Seven, Eight, NINE!  NINETEEN! Tw-tw-two\nTWO! Two! TWO!! T-t-T-TW-\nTwo. TTTWW\nTTTTWW TT TTWTW T T\nTw T Tw Tww Tw\ntw Tw T w\nW WWTW WTWT W\ntttwwwwt\nww tww\nwtttt\nt t ttt tt tw t\nMmmm\nmm mm mm\nMM MMMMM\nm m mmm m\nGgggg\nggg ggg g\nGGGGG\ng GGG\n   G GG G\n Ggg G",
                "is_finetuned": false
            },
            {
                "model_name": "meta-llama/Llama-2-7b-hf",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Penthouse\" to be performed by Kelsea Ballerini using the following genres: Pop, Country/Pop.\n\n### Lyrics:\n",
                "responses": "```\nI'm on the top floor, you're on my mind\nYou've been on this floor for too long\nNow I'll be the one to leave\nBut I don't know if I can\nCause you are the only one\nThat I want to wake up with\nAnd you don\u2019t even know my name\nIf you knew my heart\nWould you still let me stay?\nWe could go down to the roof\nOr I could stay here all night\nLet me in, let's make a toast\nTo all the times we'd had\nWhen we were younger\nDidn'\nt mean that much to me\n'Cuz now I know you better\nSo I guess I gotta leave...\nBaby, I think it'a time to go\nDon'cha know it ain'ta easy\nKnowing you like I do\nCan'nt say I won'na miss you\nNever thought I would\nOh, but I will\nYeah, oh, yeah, baby, it was nice\nIt was so nice, so good, we had our time\nSweet, sweet, never thought it would end\nNo, no, doncha think I shoulda known? Yeah,\nWhat'chu gonna do? What'cho gon'say? I\nThink I\u2019ll go, go to bed\nTake a little time, take a deep breath\nDo you really want me here? Oh, what'ya gonnado? Do you want the truth? Or do you just want someone to blame? It'sa you, not me... Oh no\nHow'bout we start over? We can take it slow\nLay down here on that bed, get it on, make love\nAll night, all day, every day... I wanna stay\nWhy'chas got to run? Why'chos got\nto go?... 'Cus I need you more\nGotta get out, got ta get away\nNeed to let you go... But I still love you... Baby, why'das got'o be this way? But you won\u2019ta stay... No, 'cause I love'y\n(I love ya)\nHad to get up, had to face the day\nGoing back to work, doing things my way\nStill, still, can'tsa be that way... So I say\nGoodbye... Goodbye\nMmm... You'r my love, my life, the air I breathe\nWithout you I ain\u2019ts a living\nWho'ds ganna love me? Who'da be there for me when I needed it the most? You, of course... Why\u2019d you have to make me leave? Mmm\nP.S. I hope you\u2019re happy\n...Happy... Happy\n[Outro]\n",
                "is_finetuned": false
            },
            {
                "model_name": "grantsl/LyricaLlama",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Penthouse\" to be performed by Kelsea Ballerini using the following genres: Pop, Country/Pop.\n\n### Lyrics:\n",
                "responses": "I'm a woman with a plan\nAnd a man who's got a way with his hands\nSo it'd be easy to fall\nInto the arms of a stranger\nBut I've got my own agenda\nIt'll be a while before I settle down\nGotta keep on moving\nKeep on running around\n'Til I get to the top\nOf this penthouse\nTop of this, pent house\nOh, yeah, oh, I\nCan't wait to get there\nWhere the view is so clear\nFrom the pent, from the, pen house, house (Oh)\nYeah, you know, that'\ns where I wanna be\nAt the very top of the world\nLooking down on everybody else\nThat' s where, where my heart is\nWhen I think of my future\nDon' t wan na settling down, settlin down (No) , no\nJust gotta keep moving (Moving) and keep running (Running) around (Around) 'Till I reach the\ntop (Top) of that pent (Pen) house househouse (House) Yeah, Oh\nYou know that that is where\nMy heart wants to stay\nMaking memories with the man that I love\nSitting on the balcony of our pent\nhouse, our pen, top house House (Ooh) Oh, Yeah\nHey, Hey\nOur pent-house house-ouse house Oh yeah\n(Ow) Hey, hey\nPicturing my life with you\nWith you, with, uh, my love, love (Love) My love love Yeah (Yay) You know (Know) That' that\nis where (Where) I want to (Want) be (Be) At the (At) very (Very) top (Of) the wo-orld (World) Lookin' down o-on (On) everybody (Everybody) else (Else) Tha-at' sa-a where' my, heart (Heart) Is when (When) Thinkin', thinkin ' about my (My) future (Future) Don' ta settl-ing (Settl) down down No, just got to keep (Kept) moving moving and (And) keep runnin run-ning around 'till (T-T) reach (Reach) that top top pent pent hous house hous-e (house) House house yeah (yeah) you (You) know tha that (That) is whe-re my he-art (heart) wants (wants) to sta-ay (Stay, stay) Making (Memories) memori-es (memories, memo-ries) with (With) my man (Man) man and sit-ting on (on) our bal-con-ey of (of) ou-r (our) pent pen hous (hous) hous hous\ne houseyeaah (ooh-oah-ahh) heahea (oh yeah) ooyeaaah\nhey (heyy) yea\nyeayah yeaaaay\nooooohh\nohhhhh\neeeeehhhh heey\nyou know\nthat' thar is whaat I waant\nto be at the verrry top o",
                "is_finetuned": true
            },
            {
                "model_name": "grantsl/LyricaLlama",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Penthouse\" to be performed by Kelsea Ballerini using the following genres: Pop, Country/Pop.\n\n### Lyrics:\n",
                "responses": "I'm not looking for love\nBut I can't help what I feel\nIt's like I've been waiting for you\nMy whole life\nAnd I know that it'll take some time\nTo let you in, but I think I am\nCause I don' want to let this go\nNo I never want this to stop\nYeah I could stay here forever\nRight here in your arms\nWe're like a house on fire\nYou'd think that we'da burned out\nBy now but it just keeps getting hotter\nThe more that I get to know you, know\nThat I, I\nDon' know where this is going\nJust know it feels like home\nBaby, it might be crazy\nMaybe we should just slow down\nOh but, baby, you don know me\nIf I ever wanna slow it down, down. down (Oh)\nAin' t gonna happen\nSo let' s just go up\nUp to the penthouse\nLet' S just stay up here\nUntil the sun goes down in the sky\nOn the roof top, on the rooftop\nWhere we can see the whole town\nCan you imagine anything better than this?\nThis is what love is\nWaking up with you here by my side\nKissing you goodbye\nFeels like the end of the world\nThen we go back home and it all starts over again\nHey, hey, oh\nLook at us\nAll we do is love, love each other\nLove, loving eachother\n(Oh, ooh, yeah) Yeah\nGot a pent house view\nOf the city that you love so much\nYour arms are open wide\nLike a sky full of stars\nOpen wide like your heart\nSay you wan' stay, stay\nStay up in this pent up, pent-up, penthouse\nWith me, with me tonight\nIn this p-p-penthouse with us, us tonigh\nTonight, ton-ight (Ya, ya, ah) , yeah\nOoh\nP-Penthouses with the lights on\nNever wann go home, never wana go, go (Noo) home (Home) (No) no\nBoy we could lay here all night\nAs long as you' re here with (With) me (Me) here (Here) with, here,\nHere, her\u0435 with m\u0435 (OOhh) ooohh\nWhoa\nDamn, damn\nWhen I look at you I just can t help but smile\nEvery time I hear your voice I melt\nFor the first time in my life I got it figured out (I got figured\nout) And it s you and me and nobody else\nMaking it up as we, we live it out, out here on top\nIs this real? Is this what it feel like? (Is it real?)\nForever' is a long time but right now\nWell, this feels right, feels just like\nHome\nWhy don t you come with? Come with\nCome with up to my pent, up-to my penn-house, house\nEven though I told you we don have to\nDo you really want to go? Go? Yeah, no, let s go-go-\nGo",
                "is_finetuned": true
            },
            {
                "model_name": "grantsl/LyricaLlama",
                "prompt": "You are a creative, world-famous expert lyricist. Write lyrics for a song, given just a title, artist name, possible genres, and any additional information provided. Write lyrics for a song titled \"Penthouse\" to be performed by Kelsea Ballerini using the following genres: Pop, Country/Pop.\n\n### Lyrics:\n",
                "responses": "I'm gonna paint the walls with roses\nSleep on silk sheets for hours\nMake my own rules\nPut my name on the door\nGot a penthouse in the city\nWalkin' distance from the shows\nAnd a brand new life\nWith a view that's mine\nA pent house in this town\nOh, I'll be a star\nYeah, that is what I am\n'Cause I got a plan\nTo make it to the top\nWhere the world will be my stage\nIt'd be heaven if I could\nLive in a castle in Spain\nBut I can't, so\nFor now I guess I will have to\nLiv\u0435 in my dreams\nIn my pent ho u se in New York City\nNever wake up\nTill I get my big break\nThen I swear I won' t need\nAnything else\nNo, not a thing\nJust a little place to hang my hat\nWhen I come home from work\nThat' s all I need, yeah\nMy pent hou se, pent hous\u0435, my New Yo rk City pent\nHouse\n(That is where I wanna be)\nPen house, pen house\nLa la la, la-la-lah\nNew York, New york\nLook at me, look at what you've done\nYou' ve made me a big star, a real big\nStar\nNow I live in pent houses in L. A. and\nLondon\nThey're the best, they' re my favorite\nThings\nEvery day I make my way to work in\nVersace\nThis is how it feels, this is all that\nMatters\nThere' ll be no more sleepin', no\nMore dreamin\nAll I ever wanted, all my\nDreams are coming true\nSo I say\nThank you, thank you\nFrom the bottom of my heart\nCan'T believe I made it this far\nMaking me so proud\nOf the girl that I used to know\nShe was livin in her dream\nWho would'Ve thought\nWe' d be here now\nMe and my, me and m\u0435\nBig pent-house\nOoh, ooh\nWell, well, who would have thought we'D be\nHere now?\nBe here, be he re\nRight here\nThe pent, the pent...\nYes, we are\nAt the to p of the wor ld\nFinally\nUh-huh\nFeelin' good, feelin good\nGood, good to me\nHa, ha, ah\nAh, oh, hey, wow\nFresh off the plane, right back in that pentho use\nBaby, baby, here we go\nBack in tha penthous\u0435 again\nGet back to where we once belonged\nCome on, come on\nLet'S get back\nHome to my pen tho u s e\nAgain\nAnother day in paradise\nLike we used\nto know, like we use\nd to go, uh, you and me in our\nParadise, in o ur pent th o u\ns e,\nin our pentthous e (in my penthou\nse) , in m y penthou se\n[Outro]\nOne more time for my girl\n",
                "is_finetuned": true
            }
        ]
    }
}

### Purpose

My goal for this project was to fine-tune a large language model (LLM) on lyrics from songs to make the model better at producing lyrics.  LLM's are notorius for not necessarily being the best at creativity.  They often end up parroting what they are trained with.  However, I wanted to see if finetuning an LLM would improve the results.

### Preparation

In order to accomplish this task, I first had to collect data as well as select an LLM to train.  For song data, I used a subset of songs from a dataset on [kaggle](https://www.kaggle.com/datasets/neisse/scrapped-lyrics-from-6-genres/).  This dataset contained lyrics from 79 musical genres, collected from a Brazilan website.  For a model, I chose to use [meta-llama/Llama-2-7b-hf](https://huggingface.co/meta-llama/Llama-2-7b-hf) on huggingface.  I felt this model provided a solid foundation while also being small enough to effectively train given my hardware limitations.

### Data Preparation and Execution

I first had to prepare the data for use in finetuning.  The Dataset was heavily skewed towards Portugese music, since the source was a Brazilian website, however the majority of the songs were still in English.  I decided to only keep English songs to simplify things.  However, this still left me with 191,814 songs to work with.  Initially I hoped to train on all of these songs, however it quickly became apperant that this would not be feasible.  I ended up only using 25% of the English songs for training, testing, and validation.  This resulted in a training set of 34,667 songs.  I felt that this provided enough variety in music while also being small enough to train in a reasonable amount of time.

Building out the framework was pretty straightforward.  I ended up using a 4-bit quantized version with LoRA for finetuning the model.  The biggest issues I had during setup stemmed from versioning of dependencies. I particularly had issues getting bits-and-bytes to work with the other versions I was using.  After several attempts and itereations, I was able to build an environment that worked with all of the dependencies.

### Training

Before training at scale, I trained on a very small subset to ensure that the whole process worked properly.  Once I was satisfied with the pipeline I scripted it and tried running it.  In the process I upped the batch size which ended up causing the process to hang, something I have never seen before.  I did quite a bit of troubleshooting before I realized I could mitigate the issue by reducing the batch size once again.  Once it started running, I didnt have many issues, except for some issues with the server ending the process due to "inactivity". Luckily I had added checkpoints to the training script so I was able to recover from these crashes.  I ended up running the training for 2 full epochs over the course of 3 days.

### Results

I did not end up having as much time for testing the finetuned model as I would have liked.  However, I did conduct a simple test were I generated 3 versions of a song using the original Llama-7b and 3 versions of the same song using the finetuned LyricaLlama for 6 different songs.  3 of these songs were popular and existed before Llama-7b and the dataset existed.  The other 3 are new songs from this year which would not be in any of the training data.  I got a few people to go through and rank the results of each song.    

As a note, some results contain explicit language.  The original dataset was not filtered for explicit language

| Song_title               |   Favored_Base (0-2) |   Favored_Lyrical (3-5) | Index with Most Votes |  Index with Least Votes |
|:-------------------------|---------------:|------------------:|-------:|--------:|
| All I Want For Chirstmas |              1 |                 9 |      3 (Lyrical) |       0 (Base) |
| Blinding Lights          |              4 |                 6 |      5 (Lyrical) |       0 (Base) |
| Lover                    |              3 |                 7 |      4 (Lyrical) and 5 (Lyrical) |       0 (Base) |
| Electric Touch           |              2 |                 8 |      5 (Lyrical) |       0 (Base) |
| Next Thing You Know      |              4 |                 6 |      5 (Lyrical) |       1 (Base) |
| Penthouse                |              5 |                 5 |      1 (Base) and 5 (Lyrical) |       3 (Lyrical) |

![results](./results_plot.png)



### Future Work

Further experimentation could be conducted to understand how finetuning the model has effected the outputs.  Further research could include experimenting with different artist names and/or genres to see how these effect the type of song lyrics written.  Doing so would allow us to better understand how the model is effected by both of these factors.


In fine-tuning a large language model (LLM), my approach encompassed a thorough and multi-faceted process, focusing on natural language processing. It started with research and data preparation, where I focused on sourcing relevant data and choosing the right tools, like Q-LoRA, for the task.

I then moved to exploratory data analysis (EDA) and preparing the data, ensuring it was of high quality and ready for use in machine learning models. Operational challenges, including setting up a new environment and resolving versioning issues, were part of the MLOps phase, where I ensured everything was in place for the training process.

During development, I built and tested fine-tuning scripts. I also engaged in proactive debugging during the training phase, adding checkpoints to manage potential crashes.

The training phase was iterative, requiring continuous monitoring and adjustments. It culminated in a comprehensive testing phase where I evaluated the model's improvements.

Overall, I dedicated 21.5 hours to this project, which highlights the meticulous effort and dedication required in the field of machine learning, especially in optimizing large language models.